In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, SimpleRNN, Activation, Bidirectional

Using TensorFlow backend.
/home/ansatt/vsetty/miniconda2/envs/keras-conda-27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import os
import sys
import tensorflow as tf
config = tf.ConfigProto()
#only occupy 50% of memory on the GPU
config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options.allow_growth = True
tf.Session(config=config)
os.environ['KERAS_BACKEND']='tensorflow'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [12]:
# prepare the dataset of input to output pairs encoded as integers
def prepareData():
    # load ascii text and covert to lowercase
    filename = "life-of-brian.txt"
    raw_text = open(filename).read()
    raw_text = raw_text.lower()
    # create mapping of unique chars to integers
    chars = sorted(list(set(raw_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    # summarize the loaded data
    n_chars = len(raw_text)
    n_vocab = len(chars)
    seq_length = 100
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length, 1):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
    print "Total Patterns: ", n_patterns
    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
    # normalize
    X = X / float(n_vocab)
    # one hot encode the output variable
    y = np_utils.to_categorical(dataY)
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    # pick a random seed
    return X, y, dataX, dataY, char_to_int, int_to_char, n_vocab

X, y, dataX, dataY, char_to_int, int_to_char, n_vocab = prepareData()
def generateText(model, filename, seedpattern):
    model.load_weights(filename)
#     model.summary()
    # generate characters
    for i in range(1000):
        x = numpy.reshape(seedpattern, (1, len(seedpattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in seedpattern]
        sys.stdout.write(result)
        seedpattern.append(index)
        seedpattern = seedpattern[1:len(seedpattern)]
    print "\nDone."
    
def getIntVals(strseedpattern):
    return [char_to_int[char] for char in strseedpattern]

Total Patterns:  86950


In [6]:
# define the SimpleRNN model
model = Sequential()
model.add(SimpleRNN(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
# define the checkpoint
filepath=filename+"weights-rnn-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
# fit the model
model.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
86950/86950 [==============================] - 26s - loss: 3.1251    
Epoch 2/20
86950/86950 [==============================] - 26s - loss: 3.0219    
Epoch 3/20
86950/86950 [==============================] - 25s - loss: 2.9882    
Epoch 4/20
86950/86950 [==============================] - 25s - loss: 2.9539    
Epoch 5/20
86950/86950 [==============================] - 25s - loss: 2.9237    
Epoch 6/20
86950/86950 [==============================] - 25s - loss: 2.8976    
Epoch 7/20
86950/86950 [==============================] - 25s - loss: 2.8742    
Epoch 8/20
86950/86950 [==============================] - 26s - loss: 2.8522    
Epoch 9/20
86950/86950 [==============================] - 26s - loss: 2.8333    
Epoch 10/20
86950/86950 [==============================] - 24s - loss: 2.8138    
Epoch 11/20
86950/86950 [==============================] - 25s - loss: 2.7979    
Epoch 12/20
86950/86950 [==============================] - 26s - loss: 2.7794    
Epoch 13/20
86950/86950 [

In [13]:

rnnmodel = Sequential()
rnnmodel.add(SimpleRNN(256, input_shape=(X.shape[1], X.shape[2])))
rnnmodel.add(Dropout(0.2))
rnnmodel.add(Dense(y.shape[1], activation='softmax'))
# X, y, dataX, dataY, char_to_int, int_to_char = prepareData()
strseedpattern = "all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? "
print "seed: " + strseedpattern
generateText(rnnmodel, "life-of-brian.txtweights-rnn-19-2.6691.hdf5", getIntVals(strseedpattern))

seed: all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? 
brian: whl 
aen boi ta  aoian: whe  ae ao aoo toe toe  arian: whl 
aen soe teee  ser.
mend  aa aad  aoean: whe 
ae a mene  aeiau  oh  hee  ae aoe noe toe sere the  aoian: whl 
aen noi te toe toe teet toe soete  soe 
aet noi te toe toue the soee ti tee  ao ane tou  arian: whl 
aen soe teee  ber boi toe toe 
arian: whl 
aen soe teee  ser.
mend  aa aad  aoean: whe 
ae a mene  aeiau  oh  hee  ae aoe noe toe sere the  aoian: whl 
aen noi te toe toe teet toe soete  soe 
aet noi te toe toue the soee ti tee  ao ane tou  arian: whl 
aen soe teee  ber boi toe toe 
arian: whl 
aen soe teee  ser.
mend  aa aad  aoean: whe 
ae a mene  aeiau  oh  hee  ae aoe noe toe sere the  aoian: whl 
aen noi te toe toe teet toe soete  soe 
aet noi te toe toue the soee ti tee  ao ane tou  arian: whl 
aen soe teee  ber boi toe toe 
arian: whl 
aen soe teee  ser.
mend  aa aad  aoean: whe 
ae a mene  aeiau  oh  

In [12]:
#define the LSTM model
lstmmodel = Sequential()
lstmmodel.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
lstmmodel.add(Dropout(0.2))
lstmmodel.add(Dense(y.shape[1], activation='softmax'))
lstmmodel.compile(loss='categorical_crossentropy', optimizer='adam')
lstmmodel.summary()

In [13]:
# define the checkpoint
filepath=filename+"weights-lstm-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [14]:
# fit the model
lstmmodel.fit(X, y, nb_epoch=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
86950/86950 [==============================] - 108s - loss: 3.1236   
Epoch 2/20
86950/86950 [==============================] - 107s - loss: 2.9583   
Epoch 3/20
86950/86950 [==============================] - 107s - loss: 2.8758   
Epoch 4/20
86950/86950 [==============================] - 107s - loss: 2.8059   
Epoch 5/20
86950/86950 [==============================] - 107s - loss: 2.7327   
Epoch 6/20
86950/86950 [==============================] - 105s - loss: 2.6656   
Epoch 7/20
86950/86950 [==============================] - 106s - loss: 2.6033   
Epoch 8/20
86950/86950 [==============================] - 107s - loss: 2.5458   
Epoch 9/20
86950/86950 [==============================] - 107s - loss: 2.4935   
Epoch 10/20
86950/86950 [==============================] - 107s - loss: 2.4458   
Epoch 11/20
86950/86950 [==============================] - 106s - loss: 2.4433   
Epoch 12/20
86950/86950 [==============================] - 107s - loss: 2.3837   
Epoch 13/20
86950/86950 [

In [60]:
lstmloadedmodel = Sequential()
lstmloadedmodel.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
lstmloadedmodel.add(Dropout(0.2))
lstmloadedmodel.add(Dense(y.shape[1], activation='softmax'))
strseedpattern = "all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? "
print "seed: " + strseedpattern
generateText(lstmloadedmodel, "montypython/life-of-brian.txtweights-lstm-19-2.0754.hdf5", getIntVals(strseedpattern))

seed: all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? 
frign oo the rooan 
rorp: the sese a sood oo the mooe oo 
brian: ihn  no  woe rooa oo the goor  and no'so 
brian: what?
brian: io,  ren. silate  and ne' tea...

brian: what?
minate: what?
menas #2: yhul  wiu  ant ii's sere th te tor oo the toon toe broan. shen so the rooans.
brian: ihn  oo  noe tooen  so hene  an an a gan  .rian.  rie noot neee  sha tea  ro, no. no. 
ren: fillowers: hhh  oer: mn  she  oo  no. no. io. conen: oh, yeal, thet' aen' whu.
milate: what?
brian: io,  ren. oilate: whal?
brian: i wasn to he to hot to the boonn. brian. brian. brian. brian! brian! arian!  roen' she kanter on the moont 
brian: i man tiin tou to tou tou tou 
brian: i was aot you to note to the moon  brian: ihn  oo  iiee yeu.
mendy: whal?
brian: io'  rer. pilate: whal?
brian: io'  ren. pilate: whal?
brian: io'  rer. pilate: whal?
brian: io'  ren. pilate: whal?
brian: io'  ren. pilate: whal?
brian

In [30]:
#Define stacked LSTM
stackedlstmmodel = Sequential()
stackedlstmmodel.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
stackedlstmmodel.add(Dropout(0.2))
#Second layer of LSTM
stackedlstmmodel.add(LSTM(256))
stackedlstmmodel.add(Dropout(0.2))
stackedlstmmodel.add(Dense(y.shape[1], activation='softmax'))
stackedlstmmodel.compile(loss='categorical_crossentropy', optimizer='adam')
stackedlstmmodel.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_11 (LSTM)                   (None, 100, 256)      264192      lstm_input_7[0][0]               
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 100, 256)      0           lstm_11[0][0]                    
____________________________________________________________________________________________________
lstm_12 (LSTM)                   (None, 256)           525312      dropout_13[0][0]                 
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 256)           0           lstm_12[0][0]                    
___________________________________________________________________________________________

In [18]:
# define the checkpoint
filepath=filename+"weights-lstmstacked-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [19]:
# fit the model
stackedlstmmodel.fit(X, y, nb_epoch=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
86950/86950 [==============================] - 389s - loss: 3.1022   
Epoch 2/50
86950/86950 [==============================] - 390s - loss: 2.7948   
Epoch 3/50
86950/86950 [==============================] - 394s - loss: 2.5955   
Epoch 4/50
86950/86950 [==============================] - 394s - loss: 2.4415   
Epoch 5/50
86950/86950 [==============================] - 395s - loss: 2.3106   
Epoch 6/50
86950/86950 [==============================] - 396s - loss: 2.2065   
Epoch 7/50
86950/86950 [==============================] - 394s - loss: 2.1216   
Epoch 8/50
86950/86950 [==============================] - 396s - loss: 2.0533   
Epoch 9/50
86950/86950 [==============================] - 397s - loss: 1.9872   
Epoch 10/50
86950/86950 [==============================] - 388s - loss: 1.9337   
Epoch 11/50
86950/86950 [==============================] - 386s - loss: 1.8808   
Epoch 12/50
86950/86950 [==============================] - 388s - loss: 1.8345   
Epoch 13/50
86950/86950 [

86950/86950 [==============================] - 387s - loss: 1.3821   
Epoch 30/50
86950/86950 [==============================] - 388s - loss: 1.3670   
Epoch 31/50
86950/86950 [==============================] - 385s - loss: 1.3509   
Epoch 32/50
86950/86950 [==============================] - 385s - loss: 1.3363   
Epoch 33/50
86950/86950 [==============================] - 385s - loss: 1.3302   
Epoch 34/50
86950/86950 [==============================] - 381s - loss: 1.3139   
Epoch 35/50
86950/86950 [==============================] - 380s - loss: 1.3019   
Epoch 36/50
86950/86950 [==============================] - 381s - loss: 1.2986   
Epoch 37/50
86950/86950 [==============================] - 382s - loss: 1.2903   
Epoch 38/50
86950/86950 [==============================] - 382s - loss: 1.2800   
Epoch 39/50
86950/86950 [==============================] - 381s - loss: 1.2665   
Epoch 40/50
86950/86950 [==============================] - 381s - loss: 1.2584   
Epoch 41/50
86950/86950 [===

In [62]:
X, y, dataX, dataY, char_to_int, int_to_char = prepareData()
strseedpattern = "all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? "
print "seed: " + strseedpattern
generateText(stackedlstmmodel, "montypython/life-of-brian.txtweights-lstmstacked-19-1.5586-bigger.hdf5", getIntVals(strseedpattern))

Total Patterns:  86950
seed: all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? 
brian: ahh.
oandy: oo, no. no. no. no. i mayen the modh the somans the soetch:
mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the messiah! mandy: oow, the messiah! the me

In [64]:
X, y, dataX, dataY, char_to_int, int_to_char = prepareData()
strseedpattern = "all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? "
print "seed: " + strseedpattern
generateText(stackedlstmmodel, "montypython/life-of-brian.txtweights-lstmstacked-48-1.2091-bigger.hdf5", getIntVals(strseedpattern))

Total Patterns:  86950
seed: all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? 
mandy: whlt tpophsers.
reg: what?
reg: yeah. teg, reg? i was to pee iim away and gollg to she frur. where was a roogher brshes.
shepherd #1: yeah. they can seally miksle wou hot to hete fige?
brian: what?
brian: what?
brian: i make to say.
'romand  weath whoe the segit bre offl the sanitation are the froundery brian she sketch:
crowd: laughing
pilate: so wou're not to gond and get ott there the grems are the freen.
crian: well, it's not a jimtleh your sandin! arthur: the shoe os tee lessiah!
shepherd #2: yeah. i can't hear a word hor a bet on the say. there is a ban soeai to be a bit of the say, then? brean yourh: what would you are detinnt.
harry the haggler: no, no, no, no. no. no. no. i just the big of your linht bioney brian congngts. chhhus dickus the sketch:
crowd: laughing
pilate: so were the mentlered are the fromings br oi the face.
brian: well, it'

In [14]:
strseedpattern = "all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? "
print strseedpattern
bilstmmodel = Sequential()
bilstmmodel.add(Bidirectional(LSTM(256),  input_shape=(X.shape[1], X.shape[2])))
bilstmmodel.add(Dropout(0.2))
bilstmmodel.add(Dense(y.shape[1], activation='softmax'))
# X, y, dataX, dataY, char_to_int = prepareData()
generateText(bilstmmodel, "montypython/life-of-brian.txtweights-bilstm-219-1.6268.hdf5", getIntVals(strseedpattern))

all right, apart from sanitation, medicine, education, wine, what have the romans ever done for us? 
sartos: shlt se lotel  brian: i'l not the missiah, so  shan'm jom thet ha se have the bant rine thet  thet so nevel  ali no hte all drant mf tai lote thin the segn th liver th the roran the lirt sf line, she 
reg: ih, yeah. thet'r nose thet soeat oo the foter as an thes.

rere mil te teoe to tean they  brian. i was 'rou dn these in thes, ie san gom thes thee, hh so tos toenk of eare, a ban aam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam bam ba